## Πρακτική Εφαρμογή: Ανίχνευση Απειλών και Ανωμαλιών στο Σκοτεινό Διαδίκτυο

---

Loading and Concatenating the CSV Files

In [ ]:
import os
import pandas as pd
from zipfile import ZipFile

# Define paths
zip_file_path = 'path_to_your_zip_file/MachineLearningCSV.zip'  # Change this to your local path
extract_dir = 'path_to_extracted_files/cicids2017/'  # Change this to your desired extraction path

# Extract the ZIP file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List CSV files in the directory
csv_files_path = os.path.join(extract_dir, 'MachineLearningCVE')
csv_files = os.listdir(csv_files_path)

# Load and concatenate the CSV files into a single DataFrame
df_list = [pd.read_csv(os.path.join(csv_files_path, file)) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Display the shape and first few rows
print(f"Dataset shape: {df.shape}")
df.head()

Data Pre-processing

In [ ]:
# Identify and retain useful columns
# Example: Drop columns that may not be useful, like IP addresses or those with too many missing values
df = df.dropna(axis=1, how='any')  # Example: Dropping columns with any NaN values
df = df.select_dtypes(include=[int, float])  # Example: Keeping only numeric columns

# Display the updated DataFrame structure
print(f"Preprocessed dataset shape: {df.shape}")
df.head()

Splitting the Dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Assume the target variable is the last column
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

Scaling Numerical Features

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and test sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Encoding Categorical Features (if any)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Example: Apply OneHotEncoder if there are categorical columns
encoder = OneHotEncoder()

# Fit and transform on training set, then transform on the test set
# Note: Adapt this code to your specific dataset
# X_train_encoded = encoder.fit_transform(X_train_categorical)
# X_test_encoded = encoder.transform(X_test_categorical)


Feature Extraction

In [ ]:
from sklearn.decomposition import PCA

# Example: Applying PCA for dimensionality reduction (optional)
pca = PCA(n_components=10)  # Example: Reduce to 10 components
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


Applying Classifiers

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize classifiers
models = {
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'K-Neighbors': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train_pca, y_train)  # Use X_train_scaled if not using PCA
    y_pred = model.predict(X_test_pca)  # Use X_test_scaled if not using PCA
    print(f"Results for {name}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))

Result Analysis